# ECON 5140: Applied Econometrics
## Lesson 1: Generalized Linear Models (GLM) - Tech Company Examples

This notebook demonstrates GLM methods using real-world examples from tech companies.

**Topics covered:**
1. Binary outcomes: Logistic and Probit regression (ad click prediction)
2. Count outcomes: Poisson regression (API usage/website visits)
3. Model comparison and interpretation


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit, Probit
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.stattools import durbin_watson

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)


## Part 1: Binary Outcomes - Ad Click Prediction

**Scenario:** A tech company wants to predict whether users will click on an advertisement based on:
- Time spent on website (minutes)
- Number of previous purchases
- User age
- Whether user is a premium member (0/1)

This is a classic binary classification problem where OLS fails because:
- Predictions can be outside [0,1]
- Errors are heteroskedastic
- We need to model probabilities, not outcomes directly


In [ ]:
# Generate synthetic data for ad click prediction
n = 2000

# Generate covariates
time_on_site = np.random.exponential(5, n)  # Time in minutes
previous_purchases = np.random.poisson(2, n)  # Count of previous purchases
age = np.random.normal(35, 10, n).clip(18, 65)  # Age between 18-65
premium_member = np.random.binomial(1, 0.3, n)  # 30% are premium members

# True coefficients (latent variable model)
# [intercept, time_on_site, previous_purchases, age, premium_member]
beta_true = np.array([-2.0, 0.5, 0.3, -0.02, 1.2])
X = np.column_stack([
    np.ones(n),
    time_on_site,
    previous_purchases,
    age,
    premium_member
])

# Generate latent variable and binary outcome
# Using probit structure: Y* = X*beta + epsilon, Y = 1 if Y* > 0
epsilon = np.random.normal(0, 1, n)
y_star = X @ beta_true + epsilon
y_binary = (y_star > 0).astype(int)

# Create DataFrame
df_clicks = pd.DataFrame({
    'clicked': y_binary,
    'time_on_site': time_on_site,
    'previous_purchases': previous_purchases,
    'age': age,
    'premium_member': premium_member
})

print("Dataset Summary:")
print(df_clicks.describe())
print(f"\nClick rate: {df_clicks['clicked'].mean():.2%}")


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 5)

### Why OLS Fails for Binary Outcomes


In [ ]:
# Try OLS on binary outcome
X_ols = sm.add_constant(df_clicks[['time_on_site', 'previous_purchases', 'age', 'premium_member']])
ols_model = sm.OLS(df_clicks['clicked'], X_ols).fit()

# Predict probabilities
ols_pred = ols_model.predict(X_ols)

print("OLS Model Results:")
print(ols_model.summary().tables[1])
print(f"\nMinimum predicted value: {ols_pred.min():.4f}")
print(f"Maximum predicted value: {ols_pred.max():.4f}")
print(f"Predictions outside [0,1]: {((ols_pred < 0) | (ols_pred > 1)).sum()} out of {len(ols_pred)}")

# Visualize the problem
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Predicted probabilities vs actual
axes[0].scatter(df_clicks['time_on_site'], ols_pred, alpha=0.3, label='OLS predictions')
axes[0].axhline(y=0, color='r', linestyle='--', label='Valid range [0,1]')
axes[0].axhline(y=1, color='r', linestyle='--')
axes[0].set_xlabel('Time on Site (minutes)')
axes[0].set_ylabel('Predicted Probability')
axes[0].set_title('OLS: Predictions Can Be Outside [0,1]')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Distribution of predictions
axes[1].hist(ols_pred, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Valid range')
axes[1].axvline(x=1, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Probability')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of OLS Predictions')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Logistic Regression (Logit Model)

The logit model uses the sigmoid function to map the linear predictor to valid probabilities:

$$P(Y=1|X) = \frac{1}{1 + e^{-X\beta}}$$

Or equivalently: $\log\left(\frac{p}{1-p}\right) = X\beta$


In [ ]:
# Fit logistic regression
X_logit = sm.add_constant(df_clicks[['time_on_site', 'previous_purchases', 'age', 'premium_member']])
logit_model = Logit(df_clicks['clicked'], X_logit).fit()

print("Logistic Regression Results:")
print(logit_model.summary())


In [ ]:
# Get predicted probabilities from logit
logit_pred = logit_model.predict(X_logit)

# Calculate marginal effects (average marginal effects)
# For logit: ME = beta * p * (1-p)
marginal_effects = logit_model.get_margeff()
print("\nAverage Marginal Effects (Logit):")
print(marginal_effects.summary())

# Visualize logit predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Predicted probabilities
axes[0].scatter(df_clicks['time_on_site'], logit_pred, alpha=0.3, c=df_clicks['clicked'], 
                cmap='RdYlGn', s=20)
axes[0].set_xlabel('Time on Site (minutes)')
axes[0].set_ylabel('Predicted Probability (Logit)')
axes[0].set_title('Logit: All Predictions in [0,1]')
axes[0].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='Decision boundary (0.5)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: S-shaped curve
sorted_idx = np.argsort(df_clicks['time_on_site'])
axes[1].plot(df_clicks['time_on_site'].values[sorted_idx], 
             logit_pred.values[sorted_idx], 'b-', linewidth=2, label='Logit prediction')
axes[1].scatter(df_clicks['time_on_site'], df_clicks['clicked'], 
                alpha=0.2, s=10, color='gray', label='Actual outcomes')
axes[1].set_xlabel('Time on Site (minutes)')
axes[1].set_ylabel('Probability of Click')
axes[1].set_title('Sigmoid Function: S-Shaped Probability Curve')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nLogit predictions range: [{logit_pred.min():.4f}, {logit_pred.max():.4f}]")


### Probit Regression

The probit model uses the standard normal CDF to map the linear predictor to probabilities:

$$P(Y=1|X) = \Phi(X\beta)$$

where $\Phi(\cdot)$ is the standard normal cumulative distribution function.

**Latent variable interpretation:** $Y^* = X\beta + \varepsilon$, where $\varepsilon \sim N(0,1)$, and $Y = 1$ if $Y^* > 0$.


In [ ]:
# Fit probit regression
probit_model = Probit(df_clicks['clicked'], X_logit).fit()

print("Probit Regression Results:")
print(probit_model.summary())


In [ ]:
# Get predicted probabilities from probit
probit_pred = probit_model.predict(X_logit)

# Calculate marginal effects for probit
probit_marginal_effects = probit_model.get_margeff()
print("\nAverage Marginal Effects (Probit):")
print(probit_marginal_effects.summary())

# Compare logit and probit
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Compare predictions
sorted_idx = np.argsort(df_clicks['time_on_site'])
axes[0].plot(df_clicks['time_on_site'].values[sorted_idx], 
             logit_pred.values[sorted_idx], 'b-', linewidth=2, label='Logit')
axes[0].plot(df_clicks['time_on_site'].values[sorted_idx], 
             probit_pred.values[sorted_idx], 'r--', linewidth=2, label='Probit')
axes[0].set_xlabel('Time on Site (minutes)')
axes[0].set_ylabel('Predicted Probability')
axes[0].set_title('Logit vs Probit: Very Similar Predictions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Scatter of logit vs probit predictions
axes[1].scatter(logit_pred, probit_pred, alpha=0.3)
axes[1].plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect agreement')
axes[1].set_xlabel('Logit Predictions')
axes[1].set_ylabel('Probit Predictions')
axes[1].set_title('Logit vs Probit: Correlation')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nCorrelation between logit and probit predictions: {np.corrcoef(logit_pred, probit_pred)[0,1]:.4f}")


### Model Comparison: Logit vs Probit

**Key takeaways:**
- Both models produce very similar predictions
- Coefficients differ in scale (probit coefficients are typically smaller)
- Marginal effects are usually very similar
- Choice rarely changes substantive conclusions


In [ ]:
# Compare coefficients (note: probit coefficients are typically smaller)
comparison = pd.DataFrame({
    'Logit': logit_model.params,
    'Probit': probit_model.params,
    'Logit_ME': marginal_effects.margeff,
    'Probit_ME': probit_marginal_effects.margeff
})
comparison['Ratio_Logit_Probit'] = comparison['Logit'] / comparison['Probit']

print("Coefficient Comparison:")
print(comparison.round(4))

# Compare model fit
print("\nModel Fit Comparison:")
print(f"Logit Log-Likelihood: {logit_model.llf:.2f}")
print(f"Probit Log-Likelihood: {probit_model.llf:.2f}")
print(f"Logit AIC: {logit_model.aic:.2f}")
print(f"Probit AIC: {probit_model.aic:.2f}")


## Part 2: Count Outcomes - API Usage Prediction

**Scenario:** A cloud service provider wants to predict the number of API calls made by customers based on:
- Subscription tier (free, basic, premium)
- Number of active users
- Days since account creation
- Monthly subscription fee

This is a count outcome problem where:
- Outcomes are non-negative integers (0, 1, 2, ...)
- Variance increases with the mean
- OLS can produce negative predictions


In [ ]:
# Generate synthetic data for API usage
n_count = 1500

# Generate covariates
subscription_tier = np.random.choice([0, 1, 2], n_count, p=[0.4, 0.4, 0.2])  # 0=free, 1=basic, 2=premium
active_users = np.random.poisson(50, n_count).clip(1, 200)
days_since_creation = np.random.exponential(365, n_count).clip(1, 2000)
monthly_fee = np.where(subscription_tier == 0, 0,
                       np.where(subscription_tier == 1, 
                                np.random.normal(50, 10, n_count),
                                np.random.normal(150, 30, n_count))).clip(0)

# True coefficients for Poisson (on log scale)
beta_count_true = np.array([3.0, 0.5, 0.8, 0.01, 0.005])
X_count = np.column_stack([
    np.ones(n_count),
    subscription_tier,
    np.log(active_users + 1),  # Log transform for better fit
    days_since_creation / 100,  # Scale for interpretation
    monthly_fee / 100
])

# Generate count outcome using Poisson
# log(mu) = X*beta, so mu = exp(X*beta)
mu_true = np.exp(X_count @ beta_count_true)
y_count = np.random.poisson(mu_true)

# Create DataFrame
df_api = pd.DataFrame({
    'api_calls': y_count,
    'subscription_tier': subscription_tier,
    'active_users': active_users,
    'days_since_creation': days_since_creation,
    'monthly_fee': monthly_fee
})

print("API Usage Dataset Summary:")
print(df_api.describe())
print(f"\nMean API calls: {df_api['api_calls'].mean():.2f}")
print(f"Variance of API calls: {df_api['api_calls'].var():.2f}")
print(f"Variance/Mean ratio: {df_api['api_calls'].var() / df_api['api_calls'].mean():.2f}")
print("(For Poisson, this should be close to 1)")


### Why OLS Fails for Count Data


In [ ]:
# Try OLS on count outcome
X_count_ols = sm.add_constant(df_api[['subscription_tier', 'active_users', 
                                       'days_since_creation', 'monthly_fee']])
ols_count_model = sm.OLS(df_api['api_calls'], X_count_ols).fit()

# Predict counts
ols_count_pred = ols_count_model.predict(X_count_ols)

print("OLS Model Results (Count Outcome):")
print(ols_count_model.summary().tables[1])
print(f"\nMinimum predicted value: {ols_count_pred.min():.2f}")
print(f"Maximum predicted value: {ols_count_pred.max():.2f}")
print(f"Negative predictions: {(ols_count_pred < 0).sum()} out of {len(ols_count_pred)}")

# Visualize the problem
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Predictions vs actual
axes[0].scatter(df_api['active_users'], ols_count_pred, alpha=0.3, label='OLS predictions')
axes[0].scatter(df_api['active_users'], df_api['api_calls'], alpha=0.2, 
                color='red', s=10, label='Actual counts')
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Valid range (≥0)')
axes[0].set_xlabel('Active Users')
axes[0].set_ylabel('Predicted API Calls')
axes[0].set_title('OLS: Can Produce Negative Predictions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Distribution of predictions
axes[1].hist(ols_count_pred, bins=50, edgecolor='black', alpha=0.7, label='OLS predictions')
axes[1].hist(df_api['api_calls'], bins=50, edgecolor='black', alpha=0.5, 
             color='red', label='Actual counts')
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('API Calls')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution: OLS vs Actual')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Poisson Regression

Poisson regression models count outcomes using:

- **Distribution:** $Y|X \sim \text{Poisson}(\mu)$
- **Link function:** $\log(\mu) = X\beta$ (canonical log link)
- **Properties:** $E[Y|X] = \mu$, $\text{Var}(Y|X) = \mu$

**Interpretation:** Coefficients are on the log-count scale. A one-unit increase in $X_k$ multiplies the expected count by $e^{\beta_k}$.


In [ ]:
# Prepare data for Poisson regression
X_poisson = sm.add_constant(pd.DataFrame({
    'subscription_tier': df_api['subscription_tier'],
    'log_active_users': np.log(df_api['active_users'] + 1),
    'days_scaled': df_api['days_since_creation'] / 100,
    'fee_scaled': df_api['monthly_fee'] / 100
}))

# Fit Poisson regression
poisson_model = GLM(df_api['api_calls'], X_poisson, family=families.Poisson()).fit()

print("Poisson Regression Results:")
print(poisson_model.summary())


In [ ]:
# Get predicted counts
poisson_pred = poisson_model.predict(X_poisson)

# Interpret coefficients
print("\nCoefficient Interpretation:")
print("=" * 60)
coef_names = ['Intercept', 'Subscription Tier', 'Log(Active Users)', 
              'Days (scaled)', 'Monthly Fee (scaled)']
for i, (name, coef) in enumerate(zip(coef_names, poisson_model.params)):
    if i == 0:
        print(f"{name}: {coef:.4f}")
    else:
        pct_change = (np.exp(coef) - 1) * 100
        print(f"{name}: {coef:.4f} (exp({coef:.4f}) = {np.exp(coef):.4f}, "
              f"{pct_change:+.2f}% change per unit)")

# Visualize Poisson model
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Predictions vs actual
axes[0].scatter(df_api['active_users'], poisson_pred, alpha=0.3, 
                label='Poisson predictions', s=20)
axes[0].scatter(df_api['active_users'], df_api['api_calls'], alpha=0.2, 
                color='red', s=10, label='Actual counts')
axes[0].set_xlabel('Active Users')
axes[0].set_ylabel('Predicted API Calls')
axes[0].set_title('Poisson: All Predictions Non-Negative')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Distribution comparison
axes[1].hist(poisson_pred, bins=50, edgecolor='black', alpha=0.7, 
             label='Poisson predictions', density=True)
axes[1].hist(df_api['api_calls'], bins=50, edgecolor='black', alpha=0.5, 
             color='red', label='Actual counts', density=True)
axes[1].set_xlabel('API Calls')
axes[1].set_ylabel('Density')
axes[1].set_title('Distribution: Poisson vs Actual')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPoisson predictions range: [{poisson_pred.min():.2f}, {poisson_pred.max():.2f}]")
print(f"Negative predictions: {(poisson_pred < 0).sum()}")


### Model Comparison: OLS vs Poisson

**Key differences:**
- OLS can produce negative predictions (invalid for counts)
- Poisson ensures non-negative predictions
- Poisson accounts for mean-variance relationship
- Poisson coefficients have multiplicative interpretation


In [ ]:
# Compare model performance
comparison_count = pd.DataFrame({
    'Actual': df_api['api_calls'],
    'OLS': ols_count_pred,
    'Poisson': poisson_pred
})

# Calculate metrics
print("Model Comparison:")
print("=" * 60)
print(f"OLS - Negative predictions: {(ols_count_pred < 0).sum()}")
print(f"Poisson - Negative predictions: {(poisson_pred < 0).sum()}")
print(f"\nOLS - Mean Absolute Error: {np.mean(np.abs(df_api['api_calls'] - ols_count_pred)):.2f}")
print(f"Poisson - Mean Absolute Error: {np.mean(np.abs(df_api['api_calls'] - poisson_pred)):.2f}")
print(f"\nOLS - R-squared: {ols_count_model.rsquared:.4f}")
print(f"Poisson - Pseudo R-squared: {poisson_model.pseudo_rsquared():.4f}")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Predictions comparison
axes[0].scatter(df_api['api_calls'], ols_count_pred, alpha=0.3, label='OLS', s=20)
axes[0].scatter(df_api['api_calls'], poisson_pred, alpha=0.3, label='Poisson', s=20)
axes[0].plot([0, df_api['api_calls'].max()], [0, df_api['api_calls'].max()], 
             'r--', linewidth=2, label='Perfect prediction')
axes[0].set_xlabel('Actual API Calls')
axes[0].set_ylabel('Predicted API Calls')
axes[0].set_title('Prediction Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals
ols_residuals = df_api['api_calls'] - ols_count_pred
poisson_residuals = df_api['api_calls'] - poisson_pred

axes[1].scatter(ols_count_pred, ols_residuals, alpha=0.3, label='OLS residuals', s=20)
axes[1].scatter(poisson_pred, poisson_residuals, alpha=0.3, label='Poisson residuals', s=20)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted API Calls')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Analysis')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Summary: GLM Framework

This notebook demonstrated the GLM framework with tech company examples:

### Key Takeaways:

1. **Binary Outcomes (Logit/Probit):**
   - OLS fails because predictions can be outside [0,1]
   - Logit and probit use nonlinear link functions to ensure valid probabilities
   - Both models produce very similar results
   - Coefficients need to be interpreted via marginal effects or odds ratios

2. **Count Outcomes (Poisson):**
   - OLS fails because predictions can be negative
   - Poisson regression uses log link to ensure non-negative predictions
   - Coefficients have multiplicative interpretation
   - Accounts for mean-variance relationship

3. **GLM Structure:**
   - All GLMs share: Linear predictor $\eta = X\beta$
   - They differ in: Distribution + Link function
   - Estimation via maximum likelihood

### When to Use Each Model:

| Outcome Type | Model | Example |
|-------------|-------|---------|
| Continuous | OLS | Revenue, prices |
| Binary | Logit/Probit | Click, purchase, default |
| Count | Poisson | API calls, visits, patents |
| Ordered categories | Ordered logit/probit | Rating scales, education levels |
| Unordered categories | Multinomial logit | Brand choice, transportation mode |

**Remember:** Choose the model based on the outcome distribution, not just convenience!


## References

- ECON 5140: Applied Econometrics, Lesson 1 - Generalized Linear Models
- Statsmodels documentation: https://www.statsmodels.org/
- Cameron, A. C., & Trivedi, P. K. (2013). *Regression Analysis of Count Data*
- Wooldridge, J. M. (2010). *Econometric Analysis of Cross Section and Panel Data*
